In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime

# Read data source: latest.db

In [2]:
# read sqlite database latest.db
kin_data_df = pd.read_sql_query("SELECT * from KIN_DATA", "sqlite:///latest.db")
kinship_codes_df = pd.read_sql_query("SELECT * from KINSHIP_CODES", "sqlite:///latest.db")
biog_main_df = pd.read_sql_query("SELECT * from BIOG_MAIN", "sqlite:///latest.db")
entry_data_df = pd.read_sql_query("SELECT * from ENTRY_DATA", "sqlite:///latest.db")
entry_codes_df = pd.read_sql_query("SELECT * from ENTRY_CODES", "sqlite:///latest.db")

In [3]:
kin_data_df.head()

,tts_sysno,c_personid,c_kin_id,c_kin_code,c_source,c_pages,c_secondary_source_author,c_notes,c_autogen_notes,c_created_by,c_created_date,c_modified_by,c_modified_date
0,224.0,1,2,180,0.0,None,None,[待考。《宋史·安惇傳》云，惇二子郊、邦。],None,TTS,20070312,None,None
1,225.0,1,13310,210,7596.0,3047,None,None,None,TTS,20070312,None,None
2,226.0,1,13311,180,0.0,None,None,None,None,TTS,20070312,None,None
3,227.0,1,119997,182,2229.0,None,None,據宋史列傳CBDB宋史分傳#2159,None,load,20121116,None,None
4,228.0,1,119998,183,2229.0,None,None,據宋史列傳CBDB宋史分傳#2159,None,load,20121116,None,None


In [4]:
biog_main_df.head()

,tts_sysno,c_personid,c_name,c_name_chn,c_index_year,c_index_year_type_code,c_index_year_source_id,c_female,c_index_addr_id,c_index_addr_type_code,...,c_mingzi_proper,c_name_proper,c_surname_rm,c_mingzi_rm,c_name_rm,c_created_by,c_created_date,c_modified_by,c_modified_date,c_self_bio
0,NaN,0,Wei Xiang,未詳,NaN,,NaN,0,NaN,NaN,...,None,,None,None,,None,None,Hongsu Wang,20211212,0
1,1.0,1,An Dun,安惇,1042.0,01,NaN,0,NaN,NaN,...,None,,None,None,,TTS,20070312,相璇,20201010,1
2,2.0,2,An Fang,安邡,0.0,01,1.0,0,100430.0,1.0,...,None,None,None,None,None,TTS,20070312,None,None,0
3,3.0,3,An Tao,安燾,1065.0,2912,3001.0,0,100658.0,1.0,...,None,None,None,None,None,TTS,20070312,BDNWH,20081018,1
4,4.0,4,Zha Dao,查道,955.0,01,NaN,0,12853.0,1.0,...,None,None,None,None,None,TTS,20070312,BDLYH,20080121,1


In [5]:
entry_data_df.head()

,tts_sysno,c_personid,c_entry_code,c_sequence,c_exam_rank,c_kin_code,c_kin_id,c_assoc_code,c_assoc_id,c_year,...,c_attempt_count,c_source,c_pages,c_secondary_source_author,c_notes,c_posting_notes,c_created_by,c_created_date,c_modified_by,c_modified_date
0,NaN,1,36,0,None,0,0,0,0,1076,...,NaN,58275.0,331,None,None,None,load,20211213,None,None
1,1.0,1,50,1,0,0,0,0,0,1073,...,NaN,0.0,None,None,None,0000000,TTS,20070312,None,None
2,2.0,3,36,1,None,0,0,0,0,0,...,NaN,7596.0,3024,None,None,0000000,TTS,20070312,None,None
3,3.0,4,28,1,None,0,0,0,0,1001,...,NaN,7596.0,None,None,None,0000000,BDLYH,20080122,None,None
4,4.0,4,36,1,11,0,0,0,0,988,...,NaN,7596.0,8501,None,None,0022937,TTS,20070312,BDLYH,20080122


## Create a dataframe for the training data

index is each person, columns are the kinship types from KINSHIP_CODES

In [6]:
# Create a dataframe for the training data. index is each person, columns are the kinship types from KINSHIP_CODES

# get the columns from KINSHIP_CODES
training_data_columns = kinship_codes_df['c_kincode'].tolist()
training_data_columns.remove(-10000)
training_data_columns.remove(-1)
training_data_columns.remove(0)
# for each column, create two columns, the the first one is "k-xxx_by", the last one is "k-xxx_by“
training_data_columns =['k_' + str(code) + '_by' for code in training_data_columns] + ['k_' + str(code) + '_dy' for code in training_data_columns]
print(training_data_columns[:5])

# add columns from entry_data, the prefix is 'e_'
entry_codes_columns = entry_codes_df["c_entry_code"].tolist()
entry_codes_columns.remove(-1)
entry_codes_columns.remove(0)
entry_codes_columns = ['e_' + str(code) for code in entry_codes_columns]
training_data_columns.extend(entry_codes_columns)
print(entry_codes_columns[:5])

training_data_columns.append('person_id')
training_data_columns.append('birth_year')
training_data_columns.append('death_year')
# remove -10000, -1 and 0

print(training_data_columns[-5:])


['k_2_by', 'k_3_by', 'k_4_by', 'k_6_by', 'k_8_by']
['e_1', 'e_4', 'e_5', 'e_7', 'e_8']
['e_345', 'e_346', 'person_id', 'birth_year', 'death_year']


### Create personid, birth year and death year columns

In [7]:
data_df = pd.DataFrame(columns=training_data_columns)

# add c_person_id and c_birth_year from biog_main_df to the dataframe
data_df['person_id'] = biog_main_df['c_personid']
data_df['birth_year'] = biog_main_df['c_birthyear']
data_df['death_year'] = biog_main_df['c_deathyear']
# Drop person_id == 0
data_df = data_df[data_df['person_id'] != 0]
data_df = data_df.fillna(np.nan)
print(data_df.shape)
data_df.head()

(535180, 1223)


,k_2_by,k_3_by,k_4_by,k_6_by,k_8_by,k_9_by,k_10_by,k_11_by,k_12_by,k_13_by,...,e_339,e_340,e_341,e_342,e_343,e_345,e_346,person_id,birth_year,death_year
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1042.0,1104.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,955.0,1018.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0.0,0.0


### If person ids are not in both KIN_DATA and ENTRY_DATA, it means that this person doesn't have any kinship relationship, we can remove them

In [8]:
#If person ids are not in both KIN_DATA and ENTRY_DATA, it means that this person don't have any kinship relationship, we can remove them
# get the person ids from kin_data_df
kin_person_ids = kin_data_df['c_personid'].tolist()
entry_person_ids = entry_data_df['c_personid'].tolist()
print(len(kin_person_ids))
print(len(entry_person_ids))
# get the union of person_ids
person_ids = set(kin_person_ids).union(set(entry_person_ids))
print(len(person_ids))

# get the intersection of person_ids and data_df['c_personid']
data_df = data_df[data_df['person_id'].isin(person_ids)]
print(data_df.shape)


537328
162621
360445
(360445, 1223)


### add birth and death year to kin_data_df

In [9]:
# add birth and death year to kin_data_df, use the c_kin_id to join biog_main_df, get the c_birthyear and c_deathyear
kin_data_df = pd.merge(kin_data_df, biog_main_df[['c_personid', 'c_birthyear', 'c_deathyear']], left_on='c_kin_id', right_on='c_personid', how='left')
# change 0 to NA


kin_data_df['c_birthyear'].replace(0, np.nan, inplace=True)
kin_data_df['c_deathyear'].replace(0, np.nan, inplace=True)

# remove the rows with NA in both c_birthyear and c_deathyear
kin_data_df = kin_data_df.dropna(subset=['c_birthyear', 'c_deathyear'], how='all')
print(kin_data_df.shape)

kin_data_df.head()

(241621, 16)


,tts_sysno,c_personid_x,c_kin_id,c_kin_code,c_source,c_pages,c_secondary_source_author,c_notes,c_autogen_notes,c_created_by,c_created_date,c_modified_by,c_modified_date,c_personid_y,c_birthyear,c_deathyear
5,229.0,2,1,75,0.0,None,None,[待考。《宋史·安惇傳》云，惇二子郊、邦。],"Auto-generated from PersonID = 1, KinCode = 180.",TTS,20070312,None,None,1,1042.0,1104.0
6,230.0,3,3000,180,7596.0,3011,None,None,None,TTS,20070312,None,None,3000,NaN,1126.0
7,231.0,3,3001,75,7596.0,3024;3029,None,None,None,TTS,20070312,None,None,3001,NaN,1098.0
16,240.0,4,13312,131,7596.0,8500,None,None,None,TTS,20070312,None,None,13312,937.0,1006.0
32,256.0,10,3022,75,0.0,None,None,None,None,TTS,20070312,None,None,3022,1035.0,1102.0


### Clean kin_data_df based on the following rules:
1. remove rows with c_kin_id = 0
2. remove rows with c_kin_id = c_personid
3. remove rows with c_personid = 0
4. remove rows with c_kin_code = 0
5. remove rows with c_kin_code = -10000
6. remove rows with c_kin_code = -1
7. if there are multiple rows with the same c_personid and c_kin_id:
   - if c_birthyear is not NA, remove all the rows with NA
   - if c_birthyear is the same, ramdomly keep one row
   - if c_birthyear is not the same, keep the row with the smallest c_birthyear
   - if all c_birthyear are NA, keep the earliest death year reocrd
   - if all c_birthyear are NA and all c_deathyear are same, keep the earliest record

In [10]:

# 1. remove rows with c_kin_id = 0
kin_data_df = kin_data_df[kin_data_df['c_kin_id'] != 0]
# 2. remove rows with c_kin_id = c_personid
kin_data_df = kin_data_df[kin_data_df['c_kin_id'] != kin_data_df['c_personid_x']]
# 3. remove rows with c_personid = 0
kin_data_df = kin_data_df[kin_data_df['c_personid_x'] != 0]
# 4. remove rows with c_kin_code = 0
kin_data_df = kin_data_df[kin_data_df['c_kin_code'] != 0]
# 5. remove rows with c_kin_code = -10000
kin_data_df = kin_data_df[kin_data_df['c_kin_code'] != -10000]
# 6. remove rows with c_kin_code = -1
kin_data_df = kin_data_df[kin_data_df['c_kin_code'] != -1]

# 5. if there are multiple rows with the same c_personid and c_kin_id:
# sort the dataframe by c_personid, c_kin_id, c_birthyear, c_deathyear
kin_data_df = kin_data_df.sort_values(by=['c_personid_x', 'c_kin_id', 'c_birthyear', 'c_deathyear'])

# remove the rows with the same c_personid and c_kin_id
kin_data_df = kin_data_df.drop_duplicates(subset=['c_personid_x', 'c_kin_id'], keep='first')

print(kin_data_df.shape)
kin_data_df.head()

(241108, 16)


,tts_sysno,c_personid_x,c_kin_id,c_kin_code,c_source,c_pages,c_secondary_source_author,c_notes,c_autogen_notes,c_created_by,c_created_date,c_modified_by,c_modified_date,c_personid_y,c_birthyear,c_deathyear
5,229.0,2,1,75,0.0,None,None,[待考。《宋史·安惇傳》云，惇二子郊、邦。],"Auto-generated from PersonID = 1, KinCode = 180.",TTS,20070312,None,None,1,1042.0,1104.0
6,230.0,3,3000,180,7596.0,3011,None,None,None,TTS,20070312,None,None,3000,NaN,1126.0
7,231.0,3,3001,75,7596.0,3024;3029,None,None,None,TTS,20070312,None,None,3001,NaN,1098.0
16,240.0,4,13312,131,7596.0,8500,None,None,None,TTS,20070312,None,None,13312,937.0,1006.0
32,256.0,10,3022,75,0.0,None,None,None,None,TTS,20070312,None,None,3022,1035.0,1102.0


In [11]:
print(data_df.shape)

(360445, 1223)


### Fill in data_df by using kin_data_df, key is c_personid

find the corresponding column by c_kin_code, fill in the first and last year columns

In [12]:
# Fill in data_df by using kin_data_df, find data_df's person_id in kin_data_df's c_personid_x,
# find the c_kin_code to fill in the corresponding column "k_{c_kin_code}_by" by c_birthyear
# find the c_kin_code to fill in the corresponding column "k_{c_kin_code}_dy" by c_deathyear

# only use the first 5% of the rows
# data_df = data_df[:int(len(data_df) * 0.05)].copy()

print(data_df.shape)

column_count = len(data_df.columns)

# for each person in data_df, find the corresponding rows in kin_data_df
for index, row in tqdm(data_df.iterrows(), total=len(data_df)):
    person_id = row['person_id']
    kin_rows = kin_data_df[kin_data_df['c_personid_x'] == person_id]
    current_column_count = len(data_df.columns)
    if current_column_count != column_count:
        print(current_column_count)
        print(data_df.columns)
        print(index)
        print(person_id)
        print(row)
        break
    for _, kin_row in kin_rows.iterrows():
        c_kin_code = kin_row['c_kin_code']
        c_birthyear = kin_row['c_birthyear']
        c_deathyear = kin_row['c_deathyear']
        if not pd.isnull(c_birthyear):
            data_df.at[index, 'k_' + str(c_kin_code) + '_by'] = c_birthyear
        if not pd.isnull(c_deathyear):
            data_df.at[index, 'k_' + str(c_kin_code) + '_dy'] = c_deathyear

# remove biog_main and kin_data to save memory
del kin_data_df
del biog_main_df
data_df.head()

(360445, 1223)


100%|██████████| 360445/360445 [02:37<00:00, 2286.03it/s]


,k_2_by,k_3_by,k_4_by,k_6_by,k_8_by,k_9_by,k_10_by,k_11_by,k_12_by,k_13_by,...,e_339,e_340,e_341,e_342,e_343,e_345,e_346,person_id,birth_year,death_year
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1042.0,1104.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,955.0,1018.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0.0,0.0


In [13]:
print(data_df.shape)

(360445, 1223)


### Fill in data_df by using ENTRY_DATA

In [14]:
# Drop c_entry_code = 0 and NA from entry_data_df
entry_data_df = entry_data_df[entry_data_df['c_entry_code'] != 0]
entry_data_df = entry_data_df[entry_data_df['c_entry_code'] != -1]
entry_data_df = entry_data_df.dropna(subset=['c_entry_code'])

# Drop c_year = 0 and NA from entry_data_df
entry_data_df = entry_data_df[entry_data_df['c_year'] != 0]
entry_data_df = entry_data_df.dropna(subset=['c_year'])

# Sort entry_data_df by c_personid, c_year, c_entry_code and get the first row
entry_data_df = entry_data_df.sort_values(by=['c_personid', 'c_entry_code', 'c_year'])
entry_data_df = entry_data_df.drop_duplicates(subset=['c_personid', 'c_year', 'c_entry_code'], keep='first')

# Fill in data_df by using entry_data_df, find data_df's person_id in entry_data_df's c_personid,
# find the c_entry_code to fill in the corresponding column "e_{c_entry_code}" by c_year
for index, row in tqdm(data_df.iterrows(), total=len(data_df)):
    person_id = row['person_id']
    entry_rows = entry_data_df[entry_data_df['c_personid'] == person_id]
    for _, entry_row in entry_rows.iterrows():
        c_entry_code = entry_row['c_entry_code']
        c_year = entry_row['c_year']
        data_df.at[index, 'e_' + str(c_entry_code)] = c_year

# remove entry_data to save memory
del entry_data_df

data_df.head()

100%|██████████| 360445/360445 [01:58<00:00, 3052.42it/s]


,k_2_by,k_3_by,k_4_by,k_6_by,k_8_by,k_9_by,k_10_by,k_11_by,k_12_by,k_13_by,...,e_339,e_340,e_341,e_342,e_343,e_345,e_346,person_id,birth_year,death_year
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1042.0,1104.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,955.0,1018.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0.0,0.0


### Remove the columns from data_df that no records are filled in

In [15]:
data_df = data_df.dropna(axis=1, how='all')
data_df.shape

(360445, 843)

## Save training data

In [16]:
# Save data_df to a csv file
# add timestamp to the file name
now = datetime.datetime.now()
data_df.to_csv('data_' + now.strftime("%Y%m%d%H%M%S") + '.csv', index=False, header=True, encoding='utf-8-sig')

## Try xgboost

In [18]:
# use xgboost to predict the birth year
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# drop birth_year nan or 0 from data_df
data_df = data_df.dropna(subset=['birth_year'])
data_df = data_df[data_df['birth_year'] != 0]

# drop person_id, birth_year
X = data_df.drop(['person_id', 'birth_year'], axis=1)
y = data_df['birth_year']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))

print("RMSE: %f" % (rmse))

mae = mean_absolute_error(y_test, preds)

print("MAE: %f" % (mae))

r2 = r2_score(y_test, preds)

print("R2: %f" % (r2))

# Save the model to a file

import pickle

pickle.dump(xg_reg, open("xg_reg_" + now.strftime("%Y%m%d%H%M%S") + ".dat", "wb"))


RMSE: 671.298512
MAE: 597.007275
R2: 0.117207


### Using GridSearchCV to find the best hyperparameters

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Define preprocessing pipeline
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Apply preprocessing to X_train and X_test
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Define XGBoost regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=123)

# Define hyperparameters grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.3, 0.6, 0.8]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_processed, y_train)

# Get best estimator from grid search
best_xg_reg = grid_search.best_estimator_

# Predictions and evaluation
preds = best_xg_reg.predict(X_test_processed)
rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("RMSE: %f" % (rmse))
print("MAE: %f" % (mae))
print("R2: %f" % (r2))


RMSE: 468.225524
MAE: 329.144974
R2: 0.570525


In [20]:
# Save the best model to a file
pickle.dump(best_xg_reg, open("best_xg_reg_" + now.strftime("%Y%m%d%H%M%S") + ".dat", "wb"))

In [21]:
# Print the best hyperparameters
print("Best hyperparameters:")
print(grid_search.best_params_)
print("Best score:")
print(grid_search.best_score_)

Best hyperparameters:
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}
Best score:
-215737.2951866115
